In [0]:
#livelossplot setup
!pip install pycm livelossplot
%pylab inline

In [0]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedShuffleSplit

from livelossplot import PlotLosses
from pycm import *

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import torchvision.transforms as transforms

def set_seed(seed):
    """
    Use this to set ALL the random seeds to a fixed value and take out any randomness from cuda kernels
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = False  ##uses the inbuilt cudnn auto-tuner to find the fastest convolution algorithms. -
    torch.backends.cudnn.enabled   = False

    return True

# device = 'cpu'
device = torch.device("cuda:0")
if torch.cuda.device_count() > 0 and torch.cuda.is_available():
    print("Cuda installed! Running on GPU!")
    device = 'cuda'
else:
    print("No GPU available!")

In [0]:
# Other Hyperparameters
seed = 42
lr = 0.01
momentum = 0.9
batch_size = 100
test_batch_size = 1000
n_epochs = 5

In [0]:
train_data_path = '/kaggle/input/acse-miniproject/train' #'/content/drive/My Drive/Logistic/train'

In [0]:
import glob
from PIL import Image
import numpy as np
import pandas as pd
import json 

#'/content/drive/My Drive/Logistic/mapping.json'
with open("/kaggle/input/acse-miniproject/mapping.json",'r', encoding='UTF-8') as f:
     load_dict = json.load(f)


train_file = glob.glob(train_data_path+'/*/images/*.JPEG')
print(len(train_file))
train_data = []
labels = []
count = 0


for f in train_file:
    img = np.array(Image.open(f))
    label_name = f.split('/')[5] #[6]
    label = load_dict.get(label_name)
    if img.shape != (64,64,3):
        img = np.stack((img,)*3, axis=-1)
    
    train_data.append(img)
    labels.append(label)
    count += 1

print(len(train_data))

In [0]:
train_data = np.array(train_data)
labels = np.array(labels)
print(train_data.shape)
print(labels.shape)

# mean = train_data.mean(axis=(0,1,2))/255
# std = train_data.std(axis=(0,1,2))/255
mean=[0.4802, 0.4481, 0.3975]
std=[0.2770, 0.2691, 0.2821]

print(mean)
print(std)

In [0]:
shuffler = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42).split(train_data, labels)

indices = [(train_idx, validation_idx) for train_idx, validation_idx in shuffler][0]
#valid_idx = list(set(train_idx).difference(set(train_dataset)))

print(len(indices[0]))
print(len(indices[1]))

In [0]:
X_train, y_train = torch.tensor(train_data[indices[0]]).float(), torch.tensor(labels)[indices[0]]
X_val, y_val = torch.tensor(train_data[indices[1]]).float(), torch.tensor(labels)[indices[1]]


X_train = X_train.permute(0,3,1,2)
X_val = X_val.permute(0,3,1,2)


print(X_train.size())
print(X_val.size())
print(y_train.size())

In [0]:
from torch.utils.data import Dataset 

class CustomImageTensorDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        """
        Args:
            data (Tensor): A tensor containing the data e.g. images
            targets (Tensor): A tensor containing all the labels
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = data
        self.targets = targets
        self.transform = transform


    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        sample, label = self.data[idx], self.targets[idx]
        #sample = sample.view(3, 64, 63).float()/255.
        if self.transform:
            sample = self.transform(sample)


        return sample, label

In [0]:
import torchvision

train_transform = torchvision.transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.RandomCrop(224, padding=8),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

valid_transform = torchvision.transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)])

train_set = CustomImageTensorDataset(X_train, y_train.long(), transform=train_transform)
valid_set = CustomImageTensorDataset(X_val, y_val.long(), transform=valid_transform)

In [0]:
#SIMPLE SET TO LOADTER
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
valid_loader = DataLoader(valid_set, batch_size=test_batch_size, shuffle=False, num_workers=0)

In [0]:
#train + validate + evaluate
def train(model, optimizer, criterion, data_loader):
    model.train()
    train_loss, train_accuracy = 0, 0
#     t = 1
    for X, y in data_loader:
        X, y = X.to(device), y.to(device)
        optimizer.zero_grad()
#         a2 = model(X)
        a2 = model(X.view(-1, 3, 224, 224)) 
        loss = criterion(a2, y)
        loss.backward()
        train_loss += loss*X.size(0)
        y_pred = F.log_softmax(a2, dim=1).max(1)[1]
        train_accuracy += accuracy_score(y.cpu().numpy(), y_pred.detach().cpu().numpy())*X.size(0)
        optimizer.step()  
        
    return train_loss/len(data_loader.dataset), train_accuracy/len(data_loader.dataset)
  
def validate(model, criterion, data_loader):
    model.eval()
    validation_loss, validation_accuracy = 0., 0.
    for X, y in data_loader:
        with torch.no_grad():
            X, y = X.to(device), y.to(device)
#             a2 = model(X)
            a2 = model(X.view(-1, 3, 224, 224))
            #a2 = model(X.view(-1, 28*28)) #What does this have to look like for our conv-net? Make the changes!
            loss = criterion(a2, y)
            validation_loss += loss*X.size(0)
            y_pred = F.log_softmax(a2, dim=1).max(1)[1]
            validation_accuracy += accuracy_score(y.cpu().numpy(), y_pred.cpu().numpy())*X.size(0)
            
    return validation_loss/len(data_loader.dataset), validation_accuracy/len(data_loader.dataset)
  
def evaluate(model, data_loader):
    model.eval()
    ys, y_preds = [], []
    for X, y in data_loader:
        with torch.no_grad():
            X, y = X.to(device), y.to(device)
            a2 = model(X)
            # a2 = model(X.view(-1, 1, 28, 28))
            #a2 = model(X.view(-1, 28*28)) #What does this have to look like for our conv-net? Make the changes!
            y_pred = F.log_softmax(a2, dim=1).max(1)[1]
            ys.append(y.cpu().numpy())
            y_preds.append(y_pred.cpu().numpy())
            
    return np.concatenate(y_preds, 0),  np.concatenate(ys, 0)

In [0]:
def exp_lr_scheduler(optimizer, epoch, lr_decay=0.1, lr_decay_epoch=2):
    """Decay learning rate by a factor of lr_decay every lr_decay_epoch epochs"""
    if epoch % lr_decay_epoch:
        return optimizer
    
    for param_group in optimizer.param_groups:
        param_group['lr'] *= lr_decay
    return optimizer

In [0]:
def train_model(model, optimizer, weight_decay = 0., bias = True):
    set_seed(seed)
#     optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay = weight_decay)
    criterion = nn.CrossEntropyLoss()

    liveloss = PlotLosses()
    for epoch in range(n_epochs):
        optimizer = exp_lr_scheduler(optimizer, epoch)
#         print(epoch)
        train_loss, train_accuracy = train(model, optimizer, criterion, train_loader)
#         print(train_loss.item(), train_accuracy.item())
        logs = {}
        logs['' + 'log loss'] = train_loss.item()
        logs['' + 'accuracy'] = train_accuracy.item()
#     print(train_loss.item(), train_accuracy.item())
        validation_loss, validation_accuracy = validate(model, criterion, valid_loader)
#         print(validation_loss.item(), validation_accuracy.item())
        logs['val_' + 'log loss'] = validation_loss.item()
        logs['val_' + 'accuracy'] = validation_accuracy.item()

        liveloss.update(logs)
        liveloss.draw()

    return model, liveloss


In [0]:
# Finetuning
import torchvision.models as models
import torch.optim as optim

model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 200)
 
if torch.cuda.is_available():
    model_ft = model_ft.cuda()
    

In [0]:
del train_file
del X_train, y_train, X_val, y_val, shuffler, indices, mean, std, train_data, labels
del img, label_name

In [0]:
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=lr, momentum=momentum, weight_decay=1e-4)
model_ft = train_model(model_ft, optimizer_ft)

In [0]:
import glob
from PIL import Image
import numpy as np
import pandas as pd
import json 
data_path='/kaggle/input/acse-miniproject/test/'
# test_transform = torchvision.transforms.Compose([
#             transforms.ToTensor(),
#             transforms.Normalize(mean=[0.4802, 0.4481, 0.3975], std=[0.2770, 0.2691, 0.2821])])
# test_dataset = torchvision.datasets.ImageFolder(
#         root=data_path,
#         transform=test_transform)
test_file = glob.glob('/kaggle/input/acse-miniproject/test/images/*.JPEG') #'/content/drive/My Drive/Logistic/train/images/*.JPEG'
print(len(test_file))
test_data = []
test_names = []
for f in test_file:
    
    img = np.array(Image.open(f))
    test_name = f.split('/')[6][5:-5] #[7][5:-5] 
    #label = load_dict.get(label_name)
    if img.shape != (64,64,3):
        img = np.stack((img,)*3, axis=-1)
    test_data.append(img)
    test_names.append(int(test_name))
print(test_names[:5])
#y_test is only used for generating dataset. it's not the real labels
y_test = np.array(test_names)
y_test = torch.tensor(y_test)
 
X_test = np.array(test_data)
X_test = torch.tensor(X_test).float()
X_test = X_test.permute(0,3,1,2)
 
test_set = CustomImageTensorDataset(X_test, y_test.long(), transform=valid_transform)
test_loader = torch.utils.data.DataLoader(
        test_set,
        batch_size=1000,
        num_workers=0,
        shuffle=False
)


In [0]:
y_pred, y_gt = evaluate(model_ft[0], test_loader)
y_pred[:5], y_gt[:5]
res = {}
for i in range(10000):
    res["test_" + str(y_gt[i]) + ".jpeg"] = y_pred[i]
res
import csv
#'/content/drive/My Drive/Logistic/resnet50.csv'
with open('/kaggle/working/resnet50.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Filename", "Label"])
    for key, item in res.items():
        writer.writerow([key, item])